<a href="https://colab.research.google.com/github/JyzMinaBF/Data_analysis/blob/main/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 此程式為簡易的探索式資料分析以及模型選用流程 (Here are a simple procedure for EDA and some models)

# Contents:

## 1. 資料讀取與處理 (Reading and processing data)

## 2. 探索式資料分析 (Explotary Data Analysis)

## 3. 模型 (Models)

## 0. 後續使用到之套件 (Packages Used in the Program)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import statsmodels.api as sm
from scipy import stats
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 資料讀取與處理 (Reading and processing data)

### (1) 資料讀取 (Reading data)

In [3]:
def read_data(data_name : str, mode : str = "colab", sep_type : str = ",") -> pd.DataFrame:
    """Help you to universally read the csv, txt and xlsx file.

    Args:
        data_name (str): The data name in Colab mode and the route name of the data in local (.csv or .txt have to be added in the end)
        mode (str, optional): ("colab","local") Defaults to "colab".
        sep_type (str, optional): The sign used to separate the entries Defaults to ",".

    Returns:
        pd.DataFrame: The data you read in DataFrame type in pandas
    """

    data = pd.DataFrame()

    if mode == "colab":
        data_dir = "/content/drive/MyDrive/" + data_name
    elif mode == "local":
        data_dir = data_name
    if data_name[::-1][:3] == "vsc":
        data = pd.read_csv(data_dir, encoding= 'unicode_escape', sep = sep_type)
    elif data_name[::-1][:3] == "txt":
        data = pd.read_fwf(data_dir)
    elif data_name[::-1][:3] == "xslx":
        data = pd.read_excel(data_dir)

    return data

data_name = ".csv"
data = read_data(data_name)
data.head()

### (2) 資料內容總整及確認(Data summary and verify)

#### i. 總整資訊(Summary)

In [4]:
data.describe(include='all')

#### ii. 反應變數確認 (Verify Response Variable)

In [ ]:
y = "" # Remember to copy the name of variable from above directly

#### iii. 確認缺失值狀況(Check if there is any NA value)

In [ ]:
def isna_test(data : pd.DataFrame) -> pd.DataFrame:
    """This is made for detecting if there is any missing value and print the rows with missing value.

    Args:
        data (pd.DataFrame): data name
    """
    if data.isna().any().any():

        mask = data.isna().any(axis=1)

        rows_with_na = data[mask]

        print("Below are rows that contains NA value.\n")
        print(rows_with_na)
        return rows_with_na

    else:
        print('There is no missing value in the DataFrame')
        return None

### (3) 資料清洗 (Data cleaning)（Not Necessary）

In [ ]:
def fill_na_methods(data : pd.DataFrame, method : str = "mean", fill : float = 0) -> pd.DataFrame:
    """We provide many kinds of methods for you to deal with missing value.

    Args:
        data (pd.DataFrame): data name

        method (str, optional): method in the list ["delcol", "delrow", "mean", "median", "mode", "fill", "reg_impute", "knn_impute", "interpolate_linear"
        , "interpolate_poly", "interpolate_spline", "bfill", "ffill"]
        delcol : delete column
        delrow : delete row
        fill : fill in a constant which defaults to 0
        reg_impute : use regression to judge (still inventing)
        knn_impute : use KNN to judge
        interpolate_method : use for ordered data or time series data to fill in with the method behind
        _fill : fill in with the value backward or forward
        . Defaults to "mean".

        fill (float, optional): the value to be filled in. Defaults to 0.

    Raises:
        KeyError: If the method in arg is not any method that is implemented in this function, raise this error

    Returns:
        pd.DataFrame: _description_
    """

    from sklearn import impute
    methods = ["delcol", "delrow", "mean", "median", "mode", "fill", "reg_impute", "knn_impute", "interpolate_linear"
    , "interpolate_poly", "interpolate_spline", "bfill", "ffill"]
    if method not in methods:
        raise KeyError("This method has not be implemented in this function.")
    if method == "delcol":
        data.dropna(axis=1, inplace=True)  # Removes columns with any missing values
    if method == "delrow":
        data.dropna(inplace=True)  # Removes rows with any missing values
    if method == "mean":
        imputer = impute.SimpleImputer(strategy='mean')  # fill in with the mean of the column
        data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    if method == "median":
        imputer = impute.SimpleImputer(strategy='median')  # fill in with the median of the column
        data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    if method == "mode":
        imputer = impute.SimpleImputer(strategy='most_frequent')  # fill in with the mode of the column
        data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    if method == "fill":
        imputer = impute.SimpleImputer(strategy='constant', fill_value=fill)  # fill in with a constant which is default to be zero
        data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    # if method == "reg_impute":
    #     # TODO: make a research of what kind of estimator can be used here and if it is proper to be used here
    #     imputer = impute.IterativeImputer(random_state=19972003)  # use regression method to impute
    #     data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    if method == "knn_impute":
        imputer = impute.KNNImputer()  # use knn method to impute
        data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    if method == "interpolate_linear":
        data.interpolate(method='linear', inplace=True) # use this for data that is time series and can be easily fill in a linear relationship
    if method == "interpolate_poly":
        data.interpolate(method='polynomial', order=2, inplace=True) # use this for data that is time series and have a polynomial relationship
    if method == "interpolate_spline":
        data.interpolate(method='slinear', inplace=True) # use this for data that is time series and want to have a smooth line
    if method == "bfill":
        data.fillna(method='bfill', inplace=True)  # Fills missing values with the next value in the column
    if method == "ffill":
        data.fillna(method='ffill', inplace=True)  # Fills missing values with the previous value in the column

    return data

## 2. 探索式資料分析 (Explotary Data Analysis)

### (1) 類型辨識(Distinguishing type)

#### i. 自動辨識 (Passively Distinguish)

In [ ]:
def dist_type(data : pd.DataFrame) -> tuple:
    """distinguish the type of variables (features) for the EDA afterwards. There are only two types, metric or categorical.

    Args:
        data (pd.DataFrame): data name

    Returns:
        tuple: a tuple with first element which is the metric variables and second elements which is categorical variables
    """

    # select only the variables that are metric type
    metric_vars = data.select_dtypes(include='number')
    cate_vars = data.select_dtypes(exclude='number')

    metric_vars = metric_vars.columns
    cate_vars = cate_vars.columns

    return (list(metric_vars), list(cate_vars))

metric_variables, categorical_variables = dist_type(data)
print("Metric:", metric_variables)
print("Categorical:", categorical_variables)

#### ii. 手動增減 (Actively Adjust)

In [ ]:
def adjust(metric_variables, categorical_variables, metric_to_cate, cate_to_metric, drop):
  for i in drop:
    if i in metric_variables:
      metric_variables.remove(i)
    elif i in categorical_variables:
      categorical_variables.remove(i)
  for i in metric_to_cate:
    metric_variables.remove(i)
  for i in cate_to_metric:
    categorical_variables.remove(i)
  for i in metric_to_cate:
     categorical_variables.append(i)
  for i in cate_to_metric:
     metric_variables.append(i)

  return metric_variables, categorical_variables

metric_variables, categorical_variables = adjust(metric_variables, categorical_variables,
                                                 metric_to_cate = [], cate_to_metric = [], drop =[])
print("Metric:", metric_variables)
print("Categorical:", categorical_variables)

### (2) 數字型變數探索(EDA for metric variables)

#### i. 變數分布之盒狀圖與直方圖(Box plot and histogram of distribution of a variable)

In [ ]:
def distribution_of_metric(data, metric_variables, save = False):
  for m in metric_variables:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    # Create the boxplot on the second subplot
    ax1.boxplot(data[m])
    ax1.set_title("Box plot of " + m)
    ax1.set_ylabel("Values")

    # Create the histogram on the first subplot
    if max(data[m])-min(data[m]) > 1000:
      ax2.hist(data[m], bins=int((max(data[m])-min(data[m])+1)/30))
    else:
      ax2.hist(data[m], bins=int((max(data[m])-min(data[m])+1)))
    ax2.set_title("Histogram of " + m)
    ax2.set_xlabel(m)
    ax2.set_ylabel("Frequency")

    if save == True:
      plt.savefig("/content/drive/MyDrive/Box plot and histogram of " + m[0:5], format = "png")
    else:
      plt.show()

distribution_of_metric(data, metric_variables)

#### ii. 變數間相關係數熱力圖(Heatmap of coefficients of correlation between metric variables)

In [ ]:
def heatmap_cc(data : pd.DataFrame, corr_type : str = "all", save : bool = False) -> None:
    """_summary_

    Args:
        data (pd.DataFrame): data_name
        corr_type (str, optional): the cc method you want to use in these function. Defaults to "all".
        save (bool, optional): save the image in your Google Drive. Defaults to False.
        # TODO: Adjust the code to not only can be used in Google Colab.

    Raises:
        KeyError: If the method in arg is not any method that is implemented in this function, raise this error
    """

    from matplotlib import pyplot as plt
    import seaborn as sns

    methods = ["pearson", "kendall", "spearman"]

    if corr_type not in methods:
        raise KeyError("This method has not be implemented in this function.")

    def corr_heatmap(corr_t):
        corr_matrix = data.corr()

        sns.heatmap(corr_matrix, annot=True, cmap='Blues', annot_kws={"size": 8})

        plt.title("Coefficients of Correlation Between Numerical Columns")

        if save == True:
            plt.savefig("/content/drive/MyDrive/Heatmap of coefficients of correlation", format = "png")
        else:
            plt.show()

    if corr_type == "all":
        for i in methods:
            corr_heatmap(i)
    else:
        corr_heatmap(corr_type)

#### iii. 變數依時間排序之折線圖(Line Plot of metric variables according to time)

In [ ]:
def time_histogram(data, metric_variables, save = False):
  for m in metric_variables:
    plt.figure()
    plt.plot(data[m].index, data[m])
    plt.title("Line Plot of " + m + "According to time")

    if save == True:
      plt.savefig("/content/drive/MyDrive/Line plot of " + m[0:5] + " according to time", format = "png")
    else:
      plt.show()

time_histogram(data, metric_variables)

### (3) 非數字型變數探索(EDA for categorical variables)

#### i. 變數內各種類出現次數條狀圖 (Bar Chart of Count of each group in a variable)

In [ ]:
def count_of_each_group(data, categorical_variables, save = False):

  for c in categorical_variables:
    gr_data = data.groupby(c)
    count_gr_data = gr_data[c].count()

    plt.figure()

    plt.bar(count_gr_data.index, count_gr_data.values)

    plt.xlabel(c)
    plt.ylabel("Frequency")
    name = 'Bar Chart of Count of ' + c
    plt.title(name)

    if save == True:
      plt.savefig("/content/drive/MyDrive/Bar Chart of Count of" + name, format = "png")
    else:
      plt.show()

count_of_each_group(data, categorical_variables)

#### ii. 變數內各種類對於反應變數之種類出現次數對應條狀圖及其卡方檢定 (Bar Chart of Frequency of Response Variable Grouped by Categorical Variables and the Chi-square Tests)

In [ ]:
def freq_by_group(data, categorical_variables, y, save=False):
    from scipy.stats import chi2_contingency

    for c in categorical_variables:
        # Compute the frequency of each binary outcome for each group
        freq = data.groupby(c)[y].value_counts().unstack()

        # Compute the proportion by dividing each value by the sum of values in the same group
        proportions = freq.div(freq.sum(axis=1), axis=0)

        plt.figure()

        # Plot the results as a bar chart with proportions
        ax = freq.plot(kind='bar', rot=0)
        ax.set_xlabel(c)
        ax.set_ylabel('Frequency')
        ax.legend(title=y, labels=reversed(set(data[y])))
        name = "Bar Chart of Frequency of " + y + " grouped by " + c
        ax.set_title(name)
        ax.tick_params(axis='x', labelrotation=90)

        # Add text annotations for proportions on each bar
        for i, column in enumerate(proportions.columns):
            for j, value in enumerate(proportions[column]):
                ax.text(i, freq.iloc[j, i] + 0.05, f'{value:.2f}', ha='center')

        if save:
            plt.savefig("/content/drive/MyDrive/Bar Chart of Frequency of" + name)
        else:
            plt.show()

        print(freq)
        # Perform the chi-square test
        chi2, p_value, dof, expected = chi2_contingency(freq)

        # Print the results
        print(f"Chi-square statistic: {chi2:.2f}")
        print(f"P-value: {p_value:.2f}")
        print(f"Degrees of freedom: {dof}")
        print("Expected frequencies:")
        print(expected)

freq_by_group(data, categorical_variables, y)

#### iii. 變數內各種類於反應變數之平均與總計條狀圖 (Bar Chart of Mean and Sum of Each Group in Variables)

In [ ]:
def mean_and_total_of_each_group(data, categorical_variables, y, save = False):

  for c in categorical_variables:
    gr_data = data.groupby(c)

    mean_gr_data = gr_data[y].mean()
    plt.figure()
    plt.bar(mean_gr_data.index, mean_gr_data.values)
    plt.set_xlabel(c)
    plt.set_ylabel(y)
    plt.set_title('Bar Chart of Mean of ' + y + ' by ' + c)

    if save == "True":
      plt.savefig("/content/drive/MyDrive/" + 'Bar Chart of Mean and Sum of ' + y + 'by ' + c, format = "png")
    else:
      plt.show()

mean_and_total_of_each_group(data, categorical_variables, y)

## 3. 模型 (Models)

### (0) 類別型變數轉換為虛擬變數 (Changing Categorical Variables into Dummy Variables)

In [ ]:
def dummyChange(data, y, drop, clf, pos_name = ""):
  data_processed = data.drop(drop, axis = 1)
  data_processed = data_processed.drop(y, axis = 1)
  if y in categorical_variables:
    categorical_variables.remove(y)
  for i in drop:
    if i in categorical_variables:
      categorical_variables.remove(i)
  if clf == True:
    res = []
    for i in data[y]:
      if i == pos_name:
        res.append(1)
      else:
        res.append(0)
    res_dict = {y : res}
    dummy_x = pd.concat([pd.get_dummies(data_processed, columns = categorical_variables, drop_first = True), pd.DataFrame(res_dict)], axis = 1)
  else:
    dummy_x = pd.concat([pd.get_dummies(data_processed, columns = categorical_variables, drop_first = True), data[y]], axis = 1)
  return dummy_x

data_changed = dummyChange(data, y, drop = [], clf = False, pos_name = "")
print(data_changed.head())

### (1) 數值型預測 (Regression Model)

#### i. 數值型模型所需套件 (Packages for Regression Model)

In [ ]:
import time
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, RidgeCV, Ridge, PoissonRegressor
from statsmodels.tools.tools import pinv_extended
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras

#### ii. 迴歸 (Regression)

##### (i) 線性迴歸 (Linear Regression)

In [ ]:
def lm(data, y):

  X_lin = sm.add_constant(data.drop(y, axis = 1))
  model = sm.OLS(data[y], X_lin).fit()
  print("Linear regression summary:\n", model.summary(), "\n")

  model = LinearRegression()
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
                       return_train_score=True)
  print("Linear Regression RMSE:", -1 * np.mean(scores["test_neg_root_mean_squared_error"]))
  print("Linear Regrssion R Squared:", np.mean(scores["test_r2"]))

lm(data_changed, y)

##### (ii) Lasso迴歸 (Lasso Regression)

In [ ]:
def LassoReg(data, y):
  start_time = time.time()
  model = make_pipeline(StandardScaler(), LassoCV(cv=10)).fit(data.drop(y, axis = 1), data[y])
  fit_time = time.time() - start_time

  lasso = model[-1]
  ymin, ymax = lasso.mse_path_.mean(axis=-1).min() * 0.5, lasso.mse_path_.mean(axis=-1).max() * 1.5
  plt.semilogx(lasso.alphas_, lasso.mse_path_, linestyle=":")
  plt.plot(
      lasso.alphas_,
      lasso.mse_path_.mean(axis=-1),
      color="black",
      label="Average across the folds",
      linewidth=2,
  )
  plt.axvline(lasso.alpha_, linestyle="--", color="black", label="alpha: CV estimate")

  plt.ylim(ymin, ymax)
  plt.xlabel(r"$\alpha$")
  plt.ylabel("Mean square error")
  plt.legend()
  _ = plt.title(
      f"Mean square error on each fold: coordinate descent (train time: {fit_time:.2f}s)"
  )

  X_lin = sm.add_constant(data.drop(y, axis = 1))
  model = sm.OLS(data[y], X_lin)
  lasso_sm = model.fit_regularized(alpha = lasso.alpha_, L1_wt = 1)
  pinv_wexog,_ = pinv_extended(X_lin)
  normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
  final = sm.regression.linear_model.OLSResults(model,
                                              lasso_sm.params,
                                              normalized_cov_params)
  print("Lasso summary:\n", final.summary(), "\n")

  model = Lasso(lasso.alpha_)
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
                       return_train_score=True)
  print("Lasso RMSE:", -1 * np.mean(scores["test_neg_root_mean_squared_error"]))
  print("Lasso R Squared:", np.mean(scores["test_r2"]))

LassoReg(data_changed, y)

##### (iii) Ridge迴歸 (Ridge Regression)

In [ ]:
def RidgeReg(data, y):
  start_time = time.time()
  model = make_pipeline(StandardScaler(), RidgeCV(cv=10)).fit(data.drop(y, axis = 1), data[y])
  fit_time = time.time() - start_time

  ridge = model[-1]
  # ymin, ymax = ridge.mse_path_.mean(axis=-1).min() * 0.5, ridge.mse_path_.mean(axis=-1).max() * 1.5
  # plt.semilogx(ridge.alphas_, ridge.mse_path_, linestyle=":")
  # plt.plot(
  #     ridge.alphas_,
  #     ridge.mse_path_.mean(axis=-1),
  #     color="black",
  #     label="Average across the folds",
  #     linewidth=2,
  # )
  # plt.axvline(ridge.alpha_, linestyle="--", color="black", label="alpha: CV estimate")

  # plt.ylim(ymin, ymax)
  # plt.xlabel(r"$\alpha$")
  # plt.ylabel("Mean square error")
  # plt.legend()
  # _ = plt.title(
  #     f"Mean square error on each fold: coordinate descent (train time: {fit_time:.2f}s)"
  # )
  print("Ridge Alpha:", ridge.alpha_)
  X_lin = sm.add_constant(data.drop(y, axis = 1))
  model = sm.OLS(data[y], X_lin)
  ridge_sm = model.fit_regularized(alpha = ridge.alpha_, L1_wt = 0)

  pinv_wexog,_ = pinv_extended(X_lin)
  normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
  final = sm.regression.linear_model.OLSResults(model,
                                              ridge_sm.params,
                                              normalized_cov_params)
  print("Ridge summary:\n", final.summary(), "\n")

  model = Ridge(ridge.alpha_)
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
                       return_train_score=True)
  print("Ridge RMSE:", -1 * np.mean(scores["test_neg_root_mean_squared_error"]))
  print("Ridge R Squared:", np.mean(scores["test_r2"]))

RidgeReg(data_changed, y)

##### (iv) 卜瓦松迴歸 (Poisson Regression / Generalized Linear Model with Poisson Distribution)

In [ ]:
def glmPoi(data, y):
  glm_possion_model = sm.GLM(data[y], data.drop(y, axis=1),
                      family=sm.families.Poisson())
  model = glm_possion_model.fit()
  print(model.summary())
  model = PoissonRegressor(alpha = 0, max_iter = 10000)
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
                       return_train_score=True)
  print("GLM with Poisson RMSE:", -1 * np.mean(scores["test_neg_root_mean_squared_error"]))
  print("GLM with Poisson R Squared:", np.mean(scores["test_r2"]))

glmPoi(data_changed, y)

#### iii. 時間序列模型 (Time Series)

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(data[y].diff().dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
  print('\t%s: %.3f' % (key, value))
data[y].diff().dropna().plot()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
plot_pacf(data[y].diff().dropna());
plot_acf(data[y].diff().dropna());

##### (i) 整合移動平均自迴歸模型 (Autoregressive Integrated Moving Average, ARIMA)

In [ ]:
def arima(data, y):
  # Create the ARMAX model
  model = sm.tsa.arima.ARIMA(data[y], order=(4, 1, 3))

  # Fit the model to the training data
  results = model.fit()

  # make predictions on the training data
  y_pred = results.predict(start = 0)

  # calculate the MSE of the predictions
  mse = mean_squared_error(data[y], y_pred)
  r2 = r2_score(data[y], y_pred)

  # Print the model summary
  print(results.summary(), "\n")
  # print the MSE values
  print("ARIMA RMSE:", np.sqrt(mse))
  print("ARIMA R-squared:", r2)

arima(data_changed, y)

##### (ii) 整合移動平均自迴歸模型結合外生變數 (Autoregressive Moving Average eXogenous, ARMAX)

In [ ]:
def arima(data, y):
  # Create the ARMAX model
  model = sm.tsa.arima.ARIMA(data[y], order=(4, 1, 3), exog = data.drop(y, axis = 1))

  # Fit the model to the training data
  results = model.fit()

  # make predictions on the training data
  y_pred = results.predict(start = 0, exog = data.drop(y, axis = 1))

  # calculate the MSE of the predictions
  mse = mean_squared_error(data[y], y_pred)
  r2 = r2_score(data[y], y_pred)

  # Print the model summary
  print(results.summary())
  # print the MSE values
  print("ARIMA RMSE:", np.sqrt(mse))
  print("ARIMA R-squared:", r2)

arima(data_changed, y)

##### (iii) 結合自迴歸之迴歸模型 (Regression Model with Autoregression)

In [ ]:
def RegwithLag(data, y, lrtype, step):
  for i in range(1, step+1):
    data[y + str(i)] = data[y].shift(i)
  data = data[i:]
  if lrtype == "lr":
    lm(data, y)
  elif lrtype == "lasso":
    LassoReg(data, y)
  elif lrtype == "ridge":
    RidgeReg(data, y)
  elif lrtype == "poi":
    glmPoi(data, y)

RegwithLag(data_changed, y, lrtype = "ridge", step = 3)

#### iv. 迴歸樹 (Regression Tree)

In [ ]:
def RegressionTree(data, y):
  model = DecisionTreeRegressor(max_depth = 3, random_state=19972003)
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
                       return_train_score=True)
  result = model.fit(data.drop(y, axis=1), data[y])
  plot_tree(result)

  print("Regression Tree RMSE:", -1 * np.mean(scores["test_neg_root_mean_squared_error"]))
  print("Regression Tree R Squared:", np.mean(scores["test_r2"]))

RegressionTree(data_changed, y)

#### v. 迴歸支持向量機 (Support Vector Machine for Regression, SVR)

In [ ]:
def SvmReg(data, y):
  model = SVR(kernal = 'linear', C=1.0, epsilon=0.1)
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
                       return_train_score=True)
  print("SVR RMSE:", -1 * np.mean(scores["test_neg_root_mean_squared_error"]))
  print("SVR R Squared:", np.mean(scores["test_r2"]))

SvmReg(data_changed, y)

#### vi. 整合型模型 (Ensemble Models)

##### (i) 迴歸用隨機森林 (Random Forest for Regression)

In [ ]:
def RFR(data, y):
  model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
                       return_train_score=True)
  print("Random Forest RMSE:", -1 * np.mean(scores["test_neg_root_mean_squared_error"]))
  print("Random Forest R Squared:", np.mean(scores["test_r2"]))

RFR(data_changed, y)

##### (ii) 迴歸用輕量化梯度提昇機 (Light Gradient Boosting Machine for Regression, LightGBM for Regression)

In [ ]:
def LGBMR(data, y):
  x = data.drop(y, axis = 1)
  params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
  }

  train_data = lgb.Dataset(x, label=data[y])

  model = lgb.cv(params, train_data, nfold = 10, seed = 19972003, metrics = "rmse")
  # kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  # scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("neg_root_mean_squared_error", "r2"),
  #                      return_train_score=True)
  print("LightGBM RMSE:", np.mean(model["rmse-mean"]))
  # print("LightGBM R Squared:", np.mean(model["r2"]))

LGBMR(data_changed, y)

#### vii. 迴歸用深度神經網路 (Deep Neural Network for Regression, DNN for Regression)

In [ ]:
def DNNR(data, y):
  x = data.drop(y, axis = 1)

  X_train, X_test, y_train, y_test = train_test_split(x, data[y], test_size=0.1, random_state=19972003)
  reg_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
  ])

  reg_model.compile(loss='mse', optimizer='adam')
  reg_model.fit(X_train, y_train, epochs=100, batch_size=100, validation_data=(X_test, y_test))

  y_pred = reg_model.predict(X_test)

  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  # print the MSE values
  print("DNN RMSE:", np.sqrt(mse))
  print("DNN R-squared:", r2)

DNNR(data_changed, y)

### (2) 分類型預測 (Classification Model)

#### i.分類型模型所需套件 (Packages for Classification Model)

In [ ]:
from sklearn.linear_model import LogisticRegression
from statsmodels.tools.tools import pinv_extended
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

#### ii. 羅吉斯迴歸 (Logistic Regression)

In [ ]:
def logistic(data, y):
  # Create a logistic regression model and fit it to the training data
  model = sm.Logit(data[y], data.drop(y, axis=1)).fit()
  print(model.summary())
  model = LogisticRegression()
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("accuracy", "precision", "recall", 'f1', 'roc_auc'),
                       return_train_score=True)

  # Evaluate the performance of the model
  print("Logistic Accuracy:", np.mean(scores["test_accuracy"]))
  print("Logistic Precision:", np.mean(scores["test_precision"]))
  print("Logistic Recall:", np.mean(scores["test_recall"]))
  print("Logistic F1 Score:", np.mean(scores["test_f1"]))
  print("Logistic AUC Score:",np.mean(scores["test_roc_auc"]))

logistic(data_changed, y)

#### iii. 決策樹 (Decision Tree)

In [ ]:
def DT(data, y):
  model = DecisionTreeClassifier(max_depth = 3)
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("accuracy", "precision", "recall", 'f1', 'roc_auc'),
                       return_train_score=True)
  result = model.fit(data.drop(y, axis=1), data[y])
  plot_tree(result)

  # Evaluate the performance of the model
  print("Decision Tree Accuracy:", np.mean(scores["test_accuracy"]))
  print("Decision Tree Precision:", np.mean(scores["test_precision"]))
  print("Decision Tree Recall:", np.mean(scores["test_recall"]))
  print("Decision Tree F1 Score:", np.mean(scores["test_f1"]))
  print("Decision Tree AUC Score:",np.mean(scores["test_roc_auc"]))

DT(data_changed, y)

#### iv. 分類用支持向量機 (Support Vector Machine for Classification, SVC)

In [ ]:
def SVRCLA(data, y):
  model = SVC(kernel='linear')
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("accuracy", "precision", "recall", 'f1', 'roc_auc'),
                       return_train_score=True)

  # Evaluate the performance of the model
  print("SVC Accuracy:", np.mean(scores["test_accuracy"]))
  print("SVC Precision:", np.mean(scores["test_precision"]))
  print("SVC Recall:", np.mean(scores["test_recall"]))
  print("SVC F1 Score:", np.mean(scores["test_f1"]))
  print("SVC AUC Score:",np.mean(scores["test_roc_auc"]))

SVRCLA(data_changed, y)

#### v. 整合型模型 (Ensemble Models)

##### (i) 分類用隨機森林 (Random Forest for Classification)

In [ ]:
def RFC(data, y):
  model = RandomForestClassifier()
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("accuracy", "precision", "recall", 'f1', 'roc_auc'),
                       return_train_score=True)

  # Evaluate the performance of the model
  print("Random Forest Accuracy:", np.mean(scores["test_accuracy"]))
  print("Random Forest Precision:", np.mean(scores["test_precision"]))
  print("Random Forest Recall:", np.mean(scores["test_recall"]))
  print("Random Forest F1 Score:", np.mean(scores["test_f1"]))
  print("Random Forest AUC Score:",np.mean(scores["test_roc_auc"]))

RFC(data_changed, y)

##### (ii) 分類用輕量化梯度提昇機 (Light Gradient Boosting Machine for Classification, LightGBM for Classification)

In [ ]:
def LGBC(data, y):
  model = lgb.LGBMClassifier()
  kfold = KFold(n_splits=10, shuffle=True, random_state=19972003)
  scores = cross_validate(model, data.drop(y, axis=1), data[y], cv = kfold, scoring=("accuracy", "precision", "recall", 'f1', 'roc_auc'),
                       return_train_score=True)

  # Evaluate the performance of the model
  print("LightGBM Accuracy:", np.mean(scores["test_accuracy"]))
  print("LightGBM Precision:", np.mean(scores["test_precision"]))
  print("LightGBM Recall:", np.mean(scores["test_recall"]))
  print("LightGBM F1 Score:", np.mean(scores["test_f1"]))
  print("LightGBM AUC Score:", np.mean(scores["test_roc_auc"]))

LGBC(data_changed, y)

#### vi. 分類用深度神經網路 (Deep Neural Network for Classfication, DNN for Classfication)

In [ ]:
def DNNC(data, y):
  x = np.asarray(data.drop(y, axis = 1)).astype(np.float32)
  y = np.asarray(data[y]).astype(np.float32)

  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=19972003)
  clf_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
  ])

  clf_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])
  clf_model.fit(X_train, y_train, epochs=50, batch_size=100, validation_data=(X_test, y_test))

  y_pred_o = clf_model.predict(X_test)
  y_pred = []
  for i in y_pred_o:
    if i < 0.5:
      y_pred.append(0)
    else:
      y_pred.append(1)
  print("DNN Accuracy:", accuracy_score(y_test, y_pred))
  print("DNN Precision:", precision_score(y_test, y_pred, pos_label=1))
  print("DNN Recall:", recall_score(y_test, y_pred, pos_label=1))
  print("DNN F1 Score:", f1_score(y_test, y_pred, pos_label=1))

DNNC(data_changed, y)